In [2]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

sw_indo = stopwords.words("indonesian") + list(punctuation)

In [4]:
# Import Data

df = pd.read_csv("data/kompas.csv")
df.head()


,teks
0,Ginandjar Tetap Ditahan. Jaksa Agung Dilaporka...
1,Jakarta Dikangkangi Para Preman\nKALAU tak pun...
2,Penyimpangan di Setpres Seolah Terjadi Sekaran...
3,"Dibayarkan, Rapel Kenaikan Gaji Pegawai Pos\nK..."
4,"Stop Kekerasan, Elite agar Duduk Bersama\nSeju..."


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
tfidf = TfidfVectorizer(ngram_range=(1,2), tokenizer=word_tokenize, stop_words=sw_indo)
tfidf_matrix = tfidf.fit_transform(df.teks)

C:\Users\ACER\miniconda3\envs\jcop_ud\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['``'] not in stop_words.
  warnings.warn(


# TFIDF Similarity -> Document Similarity 

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix)
sim

array([[1.        , 0.00858328, 0.01060043, ..., 0.00856287, 0.00677808,
        0.01513341]])

In [12]:
sim.argsort()

array([[ 932, 1131, 1593, ...,  215,  144,    0]], dtype=int64)

In [14]:
df.teks[0][:200]

'Ginandjar Tetap Ditahan. Jaksa Agung Dilaporkan ke Polri\nKejaksaan Agung memutuskan untuk tetap menahan tersangka kasus korupsi, Ginandjar Kartasasmita, sampai batas waktu yang ditentukan KUHAP. Sedan'

In [13]:
df.teks[144][:200]

'Kejaksaan Agung Terbitkan Surat Penahanan Baru\nKejaksaan Agung (Kejagung) akhirnya menerbitkan surat perintah penahanan yang baru terhadap mantan Menteri Pertambangan dan Energi Ginandjar Kartasasmita'

In [15]:
df.teks[215][:200]

'Kuasa Hukum Ginandjar Bertahan di Rutan\nSejumlah kuasa hukum Ginandjar Kartasasmita hingga hari Selasa (17/4) pukul 22.00 masih bertahan di ruang tahanan (rutan) Kejaksaan Agung (Kejagung). Selasa pag'

In [16]:
df.teks[932][:200]

'Pusat Pengolahan Ikan Asin Lhokseumawe Terbakar\nPusat pengolahan ikan asin di kawasan Pasar Ikan Lhokseumawe (Aceh) terbakar bersama berton-ton stok ikan asin Sabtu (13/1) siang. Sedikitnya 25 gudang '

# Keyword Extraction

In [18]:
vocab = tfidf.get_feature_names_out()
vocab[-10:]

array(['zuniga memilih', 'zunnatul', 'zunnatul mafruhah', 'zurich',
       'zurich northholt', 'zw', 'zw suparman', 'zw tim', 'zx',
       'zx diserbu'], dtype=object)

In [27]:
tfidf_matrix[0].toarray()

array([[0.02115058, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [28]:
sorted_tfidf = tfidf_matrix[0].toarray()[0].argsort()
sorted_tfidf

array([274212, 365469, 365468, ..., 386379, 436652, 169219], dtype=int64)

In [29]:
vocab[169219]

'ginandjar'

In [35]:
[vocab[idx] for idx in reversed(sorted_tfidf[-10:])]

['ginandjar',
 'putusan',
 'penahanan',
 'hukum ginandjar',
 'kuasa hukum',
 'rusman',
 'kejaksaan',
 'hakim rusman',
 'kuasa',
 '9 april']

# Bungkus ke Function

In [36]:
def extract_keywords_tfidf(doc, tfidf, topk=10):
    matrix = tfidf.transform([doc])
    vocab = tfidf.get_feature_names_out()
    
    sorted_tfidf = matrix[0].toarray()[0].argsort()
    return [vocab[idx] for idx in reversed(sorted_tfidf[-topk:])]

In [41]:
teks = """
Berawal Dari Cita-Cita


Mulanya adalah cita-cita membangun kembali Indonesia, sebuah Indonesia baru  yang idam-idamkan. Indonesia yang kuat dan dinamis, sebuah negara di mana hukum tegak sebagai pilar dan atap pelindung yang mengayomi semua anak bangsa  dalam kesetaraan, dan roda ekonomi berputar menebarkan kemakmuran dan kesejahteraan yang berkeadilan. Indonesia dalam visi para pendiri Rumah Kepemimpinan adalah Indonesia baru di mana politik dan demokrasi yang berakal sehat menjadi wahana bagi segenap rakyat yang berdaulat untuk mengekspresikan kehendak luhurnya sebagai bangsa yang beradab. Dalam cita-cita para peletak dasar lembaga ini, Indonesia baru adalah Indonesia yang bermoral tinggi, bangsa yang hidup dalam kesantunan. Indonesia yang dirindukan adalah Indonesia yang bermartabat, Indonesia yang disegani oleh bangsa-bangsa sedunia,  sekaligus menjadi rahmat bagi umat manusia dan guru bagi seluruh alam.

Sebagai bangsa muslim terbesar di dunia, diyakini kita punya tanggung jawab lebih untuk membangun peradaban yang memanusiakan manusia. Kita harus tampil memimpin – bukan membebek – menunjukkan kesejatian bangsa pejuang yang mendedikasikan keberadaannya bagi kemanusiaan, sebagaimana amanat Allah SWT:

     Kamu adalah umat yang terbaik yang dilahirkan untuk manusia, menyuruh kepada yang ma’ruf, dan mencegah dari yang munkar, dan beriman kepada Allah. Sekiranya ahli Kitab beriman, tentulah itu lebih baik bagi mereka. Di antara mereka ada yang beriman, dan kebanyakan mereka adalah orang-orang yang fasik. [QS. ‘Ali Imran, 3: 110]

Agar mampu mengemban amanat ilahiyah dan kemanusiaan tersebut kita harus melakukan proses transformasi struktural dan kultural, dimulaidengan membangun paradigma baru kehidupan berbangsa dan bernegara yang mengedepankan moralitas dan penegakan hukum, kesantunan politik beretika, kematangan dan kearifan berdemokrasi, serta kemandirian berekonomi, di samping upaya yang sungguh-sungguh untuk membangun titik temu, kebersamaan, persatuan dan sinergi di antara berbagai komponen bangsa.

Jika gagal melakukan transformasi maka bangsa ini tidak akan pernah keluar dari keterpurukkan yang hingga saat ini masih memerangkap kita. Sementara transformasi niscaya membutuhkan kehadiran sosok pemimpin yang tangguh sebagai lokomotif penggeraknya, sebagai agents of transformation.  Dengan kata lain, kualitas suatu bangsa adalah fungsi dari kualitas proses transformasinya, dan kualitas proses transformasi tersebut merupakan cerminan dari kualitas kepemimpinan para pemimpinnya.

Dari seluruh komponen bangsa, pemuda, khususnya mahasiswa diyakini memiliki potensi lebih untuk dikembangkan menjadi lokomotif perubahan tersebut. Pendidikan tinggi yang dijalaninya membuat mahasiswa memiliki modal lebih besar untuk mengalami mobilitas sosial vertikal, yang pada akhirnya akan mengantarkan mereka ke posisi-posisi strategis sebagai penentu kebijakan.

Dewey (1964) mengatakan: “Etimologically, the world education means just a process of leading or bringing up. When we have the outcome of the process in mind, we speak of education as shaping, forming, molding acitivity – that is as shaping into the standard form of social acitivity.”  Jelaslah bahwa peran pendidikan dalam mobilitias sosial vertikal adalah trilogi yang meliputi kendaraan (vehicle), mesin penggerak (engine), dan lintasan (track).  Pendidikanlah yang mestinya cukup besar kapasitas angkutnya (peran sebagai vehicle), cukup kuat daya dorongnya (peran sebagai engine), dan tepat arah gerak majunya (peran sebagai track), yang membuat generasi muda bangsa ini dapat menapaki piramida sosial-politik hingga ke puncaknya untuk kemudian menerima tongkat estafet kepemimpinan bangsa.

Namun dalam kehidupan masyarakat dan bangsa masih terdapat kesenjangan yang sangat besar antara idealita trilogi peran pendidikan tersebut dengan realitanya.  Akses terhadap pendidikan berkualitas, yaitu pendidikan yang mampu mengemban trilogi peran tersebut, apalagi pada jenjang pendidikan tinggi, ternyata merupakan komoditas sangat mewah yang hanya bisa dinikmati oleh kalangan yang sangat terbatas. Pada tahun 1950 baru terdapat 10 institusi pendidikan tinggi yang menjadi “kendaraan”  bagi 6.500 orang generasi muda bangsa ini. Pada tahun 1970, yang merupakan awal era regim Soeharto, jumlah lembaga pendidikan tinggi, baik yang dikelola oleh pemerintah maupun swasta, melonjak menjadi 450, dengan jumlah mahasiswa sekitar 237 ribu orang. Menjelang tahun 1990 terdapat sekitar 900 perguruan tinggi yang mewadahi sekitar 1.486.000 mahasiswa.

Pada tahun 2004 terdapat kurang lebih 2.376 perguruan tinggi, terdiri dari 81 institusi di bawah Depdiknas, 59 di bawah Depag, dan 2.236 dikelola masyarakat, dengan jumlah peserta didik lebih dari 2,5 juta orang. Angka Partisipasi Kasar (APK) pendidikan tinggi memang telah mengalami lompatan signifikan, yaitu dari 9% di tahun 1985 menjadi 12,8% di tahun 2002.  Namun dilihat dari angka absolutnya, dari sejutar 27 juta generasi muda berusia 19-24 tahun, masih terdapat lebih dari 24 juta orang yang belum dapat mengeyam bangku perguruan tinggi.  Angka-angka yang ada tentu saja akan semakin membuat kita prihatin jika variabel kualitas, apalagi kualitas kepemimpinan yang kemudian berhasil dibentuk, ikut menjadi pertimbangan.

Keprihatinan mendalam atas situasi tersebut mengerucutkan spektrum gagasan dan pemikiran yang sejak lama berkecamuk ke arah upaya untuk melahirkan sebuah institusi yang mengambil peran strategis sebagai inkubator, sekaligus akselator, kepemimpinan baru bangsa ini, mengkristal dalam dalam tiga kata:

“Inkubator Pemimpin Indonesia”

Rumah Kepemimpinan bersifat non-partisan, non-profit, terbuka, profesional, relijius, dan kebersamaan. Semua kegiatannya disampaikan kepada publik secara terbuka dan bertanggung jawab serta seluruh hasil kegiatannya dipersembahkan untuk kepentingan bangsa dan negara.

Visi kami
“Inkubator Pemimpin Indonesia”

Menjadi inkubator pemimpin masa depan Indonesia yang siap berkontribusi dalam transformasi dan pembangunan Bangsa menuju Indonesia Maju.

Misi kami
Pembangunan Pemuda
Membangun paradigma Islam dan kompetensi Peserta dan Alumni Rumah Kepemimpinan yang Rendah hati, Open mind, Moderat, dan Obyektif serta Prestatif-Kontributif.

Pemberdayaan Pemuda
Mendorong dan mengarahkan mobilitas Pemimpin Muda Rumah Kepemimpinan ke berbagai bidang dan institusi strategis.

Kepeloporan Pemuda
Membangun kepeloporan Pemimpin Muda Rumah Kepemimpinan menjadi ”problem solver” dalam kehidupan berbangsa dan bernegara.

Penguatan Lembaga
Menjadikan Rumah Kepemimpinan sebagai wadah konsolidasi Para Pemimpin untuk berkontribusi mewujudkan Indonesia dan Dunia yang lebih baik dan bermartabat.

Nilai ROMO-PKK
Nilai ini adalah nilai lembaga yang dibangun kepada seluruh peserta serta alumni Rumah Kepemimpinan. Nilai ini menggambarkan bagaimana kerangka berpikir yang dibangun bersama yakni:

Rendah hati
Open mind
Moderat
Objektif

Prestatif
Kontributif
Kolaboratif

Idealisme Kami

Betapa inginnya kami agar bangsa ini mengetahui
bahwa mereka lebih kami cintai daripada diri kami sendiri.

Kami berbangga ketika jiwa-jiwa kami gugur
sebagai penebus bagi kehormatan mereka,
jika memang tebusan itu yang diperlukan.

Atau menjadi harga bagi tegaknya kejayaan, kemuliaan,
dan terwujudnya cita-cita mereka, jika memang itu harga yang harus dibayar.

Tiada sesuatu yang membuat kami bersikap seperti ini selain rasa cinta
yang telah mengharu-biru hati kami,
menguasai perasaan kami, memeras habis air mata kami,
dan mencabut rasa ingin tidur dari pelupuk mata kami.

Betapa berat rasa di hati
ketika kami menyaksikan bencana yang mencabik-cabik bangsa ini,
sementara kita hanya menyerah pada kehinaan dan pasrah oleh keputusasaan.

Kami ingin agar bangsa ini mengetahui bahwa kami membawa misi yang bersih dan suci;
bersih dari ambisi pribadi, bersih dari kepentingan dunia, dan bersih dari hawa nafsu.

Kami tidak mengharapkan sesuatupun dari manusia;
tidak mengharap harta benda atau imbalan lainnya,
tidak juga popularitas, apalagi sekadar ucapan terima kasih.

Yang kami harap adalah terbentuknya Indonesia yang lebih baik
dan bermartabat serta kebaikan dari Allah, Pencipta alam semesta.

"""

In [42]:
extract_keywords_tfidf(teks,tfidf)

['bangsa',
 'kepemimpinan',
 'pendidikan',
 'inkubator',
 'transformasi',
 'peran',
 'kualitas',
 'indonesia',
 'trilogi',
 'pemimpin']